In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' # use cuda(gpu) if not available use cpu
#print (device)
block_size = 8 # how many intigers can the model see at once
batch_size = 4 # how many blocks can the model see at once
max_iters = 1000
eval_iters = 250
learning_rate = 3e-3
n_embd = 384 # creates a 384 attribute embedding_vector
n_layer = 4


In [3]:
with open ('wizard_of_oz.txt', 'r', encoding='utf-8') as f: # use wizard_of_oz.txt as a text and encode it with utf-8
    text = f.read() # let the script read the text
chars = sorted(set(text)) # sort all used characters in text
vocab_size = len(chars) # sets the vocab_size to the number of characters in chars

In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) } # make string of characters into intigers (full numbers)
int_to_string = { i:ch for i,ch in enumerate(chars) } # make intigers into string of characters
encode = lambda s: [string_to_int[c] for c in s] # encode the characters into the intigers
decode = lambda l: ''.join([int_to_string[i] for i in l]) # decode the intigers into characters

data = torch.tensor(encode(text), dtype=torch.long) # turns encoded text into tensors

In [5]:
n = int(0.8*len(data)) # take only 80% of the data
train_data = data[:n] # use the 80% for training
val_data = data [n:] # use the remaining 20% for validation

def get_batch(split): # defines a function called get_batch that takes a single argument split, which tells us whether to use the training or validation data
    data = train_data if split == 'train' else val_data # This selects which dataset to use: If split is 'train', use train_data.Otherwise use val_data.
    ix = torch.randint(len(data) - block_size, (batch_size,)) # Each index i represents the start of a chunk of length block_size. We subtract block_size to ensure we don’t go out of bounds.
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix]) # For each index i in ix, it slices a chunk from data of length block_size. Then stacks them into a tensor of shape (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1]for i in ix]) # This creates the target batch y, which is the same as x but shifted by one token to the right.The model will try to predict each next token in y given the corresponding token in x
    x, y = x.to(device), y.to(device)
    return x, y

In [6]:
# makes sure that torch doesnt use gradients at all so that will reduce computation, memoryusage just better for performance
@torch.no_grad() # build in pytorch decorator that disbles any gradient calculations
def estimate_loss():
    out = {}
    model.eval() # switches to evaluation mode, disables dropout
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # switches to training mode, enables dropout
    return out

In [7]:
class AsunaLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # This creates a table that turns each word into a prediction for the next word.
        # It's like saying: "If I see word X, what word should come next?"
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        #the same as token embedding but not every char has its own embed but each letter index has its own embed
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # how many decoder blocks we have running sequentialy / layers 
        self.blocks = nn.Sequential(*[Block(.(n_embd, n_head=n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        # transforming the decoders output into something that softmax can work with 
        self.lm_head = nn.Linear(n_embd, vocab_size) #language modeling head
        
    def forward(self, index, targets=None):
        # Look up each input word in the embedding table.
        # The result is a guess (logits) for the next word at each position.
        logits = self.token_embedding_table(index)  # shape: (B, T, vocab_size)

        #idx and targets are bith (B,T) tensor of intigers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        # how long is T, it creates T number in indecies and give each a different n_embd vector (a little lookup table)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,C,vocab_size)
        
        if targets is None:
            # If we're just using the model (not training), no need for a loss.
            loss = None
        else:
            # We're training the model, so calculate how wrong the guesses were.
            # First, flatten the logits and targets to make them easier to compare.
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)         # make logits 2D: (all tokens, vocab_size)
            targets = targets.view(B*T)          # flatten targets too: (all tokens)
            # Now calculate how far off the model’s guesses were.
            loss = F.cross_entropy(logits, targets)

        # Return both the guesses and the loss (loss is None during generation)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # This function makes the model keep adding words to the input.
        # Example: start with "The", it might add "cat", then "sat", etc.

        for _ in range(max_new_tokens):
            # Get the model's guess for the next word.
            logits, _ = self.forward(index)  # shape: (B, T, vocab_size)

            # Focus only on the last word's prediction.
            logits = logits[:, -1, :]  # shape: (B, vocab_size)

            # Turn the logits into probabilities (so we can pick randomly but fairly)
            probs = F.softmax(logits, dim=-1)  # shape: (B, vocab_size)

            # Randomly pick the next word using those probabilities.
            index_next = torch.multinomial(probs, num_samples=1)  # shape: (B, 1)

            # Add the new word to the end of the sequence.
            index = torch.cat((index, index_next), dim=1)  # shape: (B, T+1)

        # Return the full sequence, including the new words we added.
        return index


model = AsunaLanguageModel(vocab_size) # make the model and tell it how many tokens it can know
m = model.to(device) # put the model on GPU if available otherwise CPU

#context = torch.zeros((1,1), dtype=torch.long, device=device) # starts with a token of 0, .long cus the tokens must be integers, uses device to generate
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist()) # generate 500 new tokens using the model starting from context .generate returns a tensor of token ids, .tolist converts the ids into clear integers, decode makes the integers into characters 
#print(generated_chars)

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate) # creates AdamW optimizer instance to update the models parameters

for iter in range(max_iters): # starts a training loop that runs for the duration of max_iters (1000) each iteration performs an update step
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    xb, yb = get_batch('train') # fetches a batch of input data (xb) and target labels (yb) from training dataset (wiz_of_oz)
    logits, loss = model.forward(xb, yb)  # passes the input and target batches thrugh the model, returns logits tyhe raw output and the loss scalar comparing the predictions and targets
    optimizer.zero_grad(set_to_none=True) # clears previos gradients from the model parameters
    loss.backward() # computes gradients of the loss with respect to all model parameters
    optimizer.step() # updates model parameters using the computed gradients and the optimizer algorithm
print(loss.item()) # .item converts the tensor into a readable number 

step: 0, train loss: 2.674, val loss: 2.704
step: 250, train loss: 2.591, val loss: 2.638
step: 500, train loss: 2.551, val loss: 2.576
step: 750, train loss: 2.533, val loss: 2.561
2.1676433086395264
